## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import time
from datetime import datetime
# Import the API key.
from config import g_key

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Skoppum,NO,2021-03-07 23:46:06,59.3833,10.4167,32.00,40,18,3.44,few clouds
1,1,Guerrero Negro,MX,2021-03-07 23:46:06,27.9769,-114.0611,62.40,60,18,16.53,few clouds
2,2,Gaoua,BF,2021-03-07 23:46:07,10.2992,-3.2508,83.07,50,91,4.59,overcast clouds
3,3,Vostok,RU,2021-03-07 23:46:07,46.4856,135.8833,12.02,95,100,4.97,overcast clouds
4,4,Pevek,RU,2021-03-07 23:46:08,69.7008,170.3133,-16.06,88,100,21.99,light snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
pcities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
pcities_df.isnull()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False
10,False,False,False,False,False,False,False,False,False,False,False
12,False,False,False,False,False,False,False,False,False,False,False
13,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
673,False,False,False,False,False,False,False,False,False,False,False
674,False,False,False,False,False,False,False,False,False,False,False
676,False,False,False,False,False,False,False,False,False,False,False
678,False,False,False,False,False,False,False,False,False,False,False


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
pcities_df = pcities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pcities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Gaoua,BF,83.07,overcast clouds,10.2992,-3.2508,
5,Benguela,AO,80.58,overcast clouds,-12.5763,13.4055,
10,Port Alfred,ZA,69.01,broken clouds,-33.5906,26.8910,
12,Yaring,TH,73.80,overcast clouds,6.8662,101.3689,
13,Port Macquarie,AU,79.00,broken clouds,-31.4333,152.9167,
16,Namatanai,PG,81.10,light rain,-3.6667,152.4333,
20,Victoria,HK,66.99,overcast clouds,22.2855,114.1577,
36,Grand-Santi,GF,73.89,overcast clouds,4.2500,-54.3833,
37,Atuona,PF,79.83,scattered clouds,-9.8000,-139.0333,
38,Arraial Do Cabo,BR,75.45,light rain,-22.9661,-42.0278,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    time.sleep(0.5)
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].isnull()
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Gaoua,BF,83.07,overcast clouds,10.2992,-3.2508,
5,Benguela,AO,80.58,overcast clouds,-12.5763,13.4055,Hotel Praia Morena
10,Port Alfred,ZA,69.01,broken clouds,-33.5906,26.8910,The Halyards Hotel
12,Yaring,TH,73.80,overcast clouds,6.8662,101.3689,สถานที่จัดงานแต่ง SOFEE & DORN
13,Port Macquarie,AU,79.00,broken clouds,-31.4333,152.9167,Rydges Port Macquarie
16,Namatanai,PG,81.10,light rain,-3.6667,152.4333,Namatanai Hotel
20,Victoria,HK,66.99,overcast clouds,22.2855,114.1577,Mini Hotel Central
36,Grand-Santi,GF,73.89,overcast clouds,4.2500,-54.3833,Hotel La Villa Couacou
37,Atuona,PF,79.83,scattered clouds,-9.8000,-139.0333,Villa Enata
38,Arraial Do Cabo,BR,75.45,light rain,-22.9661,-42.0278,Pousada Porto Praia


In [40]:
# Convert the array of dictionaries to a Pandas DataFrame.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Gaoua,BF,83.07,overcast clouds,10.2992,-3.2508,
5,Benguela,AO,80.58,overcast clouds,-12.5763,13.4055,Hotel Praia Morena
10,Port Alfred,ZA,69.01,broken clouds,-33.5906,26.8910,The Halyards Hotel
12,Yaring,TH,73.80,overcast clouds,6.8662,101.3689,สถานที่จัดงานแต่ง SOFEE & DORN
13,Port Macquarie,AU,79.00,broken clouds,-31.4333,152.9167,Rydges Port Macquarie
16,Namatanai,PG,81.10,light rain,-3.6667,152.4333,Namatanai Hotel
20,Victoria,HK,66.99,overcast clouds,22.2855,114.1577,Mini Hotel Central
36,Grand-Santi,GF,73.89,overcast clouds,4.2500,-54.3833,Hotel La Villa Couacou
37,Atuona,PF,79.83,scattered clouds,-9.8000,-139.0333,Villa Enata
38,Arraial Do Cabo,BR,75.45,light rain,-22.9661,-42.0278,Pousada Porto Praia


In [44]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [45]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [46]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))